# np.roll 활용 data augmentation
 - 참조  
 https://dacon.io/competitions/official/235689/codeshare/2347?page=1&dtype=recent&ptype=pub
 https://dacon.io/competitions/official/235689/codeshare/2306?page=1&dtype=recent&ptype=pub

## np.roll -> shifted data -> new DataFrame

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from tqdm import tqdm_notebook as tn

In [78]:
train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')
test = pd.read_csv('test_features.csv')
sample_submission = pd.read_csv('sample_submission.csv')

In [79]:
TL = train_labels[['id','label']]

In [80]:
train = pd.merge(train_features,TL, how = 'left', on = 'id')

In [81]:
train.head()

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,label
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112,37
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216,37
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629,37
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013,37
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234,37


In [82]:
Label_counts = pd.DataFrame(TL['label'].value_counts())

In [83]:
TL['label'].value_counts()

26    1518
8       97
28      55
60      48
18      47
      ... 
53      13
52      12
13      12
12      12
0       12
Name: label, Length: 61, dtype: int64

# <font bord>label 개수 확인 n개보다 적으면 augmentation</font>
 1. lable 개수 확인(<n)
 2. augmentation할 label 저장(list)
 3. label 하나씩 dataframe 불러오기
 4. 불러온 data id 개수 확인
 5. id 한개당 몇개의 shifted data 만들지 계산
     - 한개당 -> int(500/id) - shift(15씩)
 6. shift data -> data frame 변환 id,labe,time등 지정
 7. all_shift에 concat하기

In [88]:
n = 300

In [89]:
#1.
def check_label(data, label, num):
    tem = data[data['label'] == label]
    if len(tem['id'].unique())<num:
        return True
    else : return False

In [90]:
# 데이터 변환
def make_train_data(data):
    train_features = data
    x_train = []

    for uid in tqdm(train_features['id'].unique()):
        temp = np.array(train_features[train_features['id'] == uid].iloc[:,2:], np.float32).T
        x_train.append(temp)

    x_train = np.array(x_train, np.float32)
    x_train = x_train[:,:,:,np.newaxis]
    
    return x_train

In [91]:
#2.
small_label = []
for i in tn(list(train_labels['label'].unique())):
    if check_label(train,i,n):
        small_label.append(i)

c:\anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [92]:
#.데이터 프레임으로 변환
def make_shifted_data(train_data, shift_num,new_id):
    shift_data =  np.roll(train_data, shift_num, axis=2)
    time = list(range(0,600))
    a = pd.DataFrame()
    a['acc_x'] = list(shift_data[0][0].reshape(1,-1)[0])
    a['acc_y'] = list(shift_data[0][1].reshape(1,-1)[0])
    a['acc_z'] = list(shift_data[0][2].reshape(1,-1)[0])
    a['gy_x'] = list(shift_data[0][3].reshape(1,-1)[0])
    a['gy_y'] = list(shift_data[0][4].reshape(1,-1)[0])
    a['gy_z'] = list(shift_data[0][5].reshape(1,-1)[0])
    a['label'] = list(shift_data[0][6].reshape(1,-1)[0])
    a['id'] = new_id
    a['time'] = time
    return a

In [93]:
total_shift_data = pd.DataFrame()

In [94]:
def change_col1(X_train):
    col_name = []
    for i in range(len(X_train.columns)):
        tm_str = '0/0'+X_train.columns[i][0] + '_' + X_train.columns[i][1]
        col_name.append(tm_str)
    X_train.columns = col_name
    return X_train

In [95]:
features = ['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']
add_col = []
for i in range(0,1):
    tm_data = train_features[train_features['id'] == i]
    for j in tn([2,4,8]):
        start_num = 0
        for k in range(1,j+1):
            split_num = int(600 * k / j)
            tm_data2 = tm_data.iloc[start_num:split_num,:]
            tm_train = tm_data2[features].groupby('id').agg(['max', 'min', 'mean','std'])
            
            for q in range(len(tm_train.columns)):
                tm_str = str(k)+'/'+str(j)+tm_train.columns[q][0] + '_' + tm_train.columns[q][1]
                add_col.append(tm_str)
            start_num = split_num

c:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [96]:
def add_FE(tm_data,df,id_):
#     print(df)
#     print(tm_data)
    features = ['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']
    for j in [2,4,8]:
        start_num = 0
        for k in range(1,j+1):
            split_num = int(600 * k / j)
            tm_data2 = tm_data.iloc[start_num:split_num,:]
            tm_train = tm_data2[features].groupby('id').agg(['max', 'min', 'mean','std'])
            tm_col = []
            for q in range(len(tm_train.columns)):
                tm_str = str(k)+'/'+str(j)+tm_train.columns[q][0] + '_' + tm_train.columns[q][1]
                tm_col.append(tm_str)
#             print('temtrain이오오오')
#             print(tm_train)
            df.loc[id_,tm_col[0]:tm_col[-1]] = list(tm_train.iloc[0,:])
            
#             print(tm_col)
#             print('id_,tm_col[0]:tm_col[-1]id_,tm_col[0]:tm_col[-1]id_,tm_col[0]:tm_col[-1]id_,tm_col[0]:tm_col[-1]')
#             print(df.loc[id_,tm_col[0]:tm_col[-1]])
#             print('tm_train.iloc[0,:]tm_train.iloc[0,:]tm_train.iloc[0,:]tm_train.iloc[0,:]tm_train.iloc[0,:]')
#             print(tm_train.iloc[0,:])
#             print('df여기부터 최ㅈ오')
#             print(df)
            tm_col = []
            start_num = split_num
#     print('마ㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏ')
#     print(df)
    return df

In [ ]:
#3 ~ 7.
features = ['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']
new_id = 1000
for i in tn(small_label):
    print('label, ',i,"진행중")
    tem_data = train[train['label'] == i]
    id_num = list(tem_data['id'].unique())
    how_many_shift = int(n/len(id_num))
    for k in id_num:
        tem_id_data = tem_data[tem_data['id'] == k]
        train_data = make_train_data(tem_id_data)
        for j in range(1,how_many_shift+1):
            shift_num = 19 * j
            shifted_data = make_shifted_data(train_data, shift_num,new_id)
#             print(shifted_data.shape)
            X_train = shifted_data[features].groupby('id').agg(['max', 'min', 'mean','std'])
            X_train = change_col1(X_train)
            for i in add_col:
                X_train[i] = 0
#             print(X_train.iloc[0,:])
            X_train = add_FE(shifted_data,X_train,new_id)                   
#             print(X_train.iloc[0,:])
            
            X_train['label'] = i
            
#             ch_col = []
#             for i in list(X_train.columns):
#                 if i[2] != '0' and i[-3:] == 'ean':
#                     ch_col.append(i)
#             X_train['std1'] = X_train[[ch_col[0],ch_col[6]]].std(1)
#             X_train['std2'] = X_train[[ch_col[1],ch_col[7]]].std(1)
#             X_train['std3'] = X_train[[ch_col[2],ch_col[8]]].std(1)
#             X_train['std4'] = X_train[[ch_col[3],ch_col[9]]].std(1)
#             X_train['std5'] = X_train[[ch_col[4],ch_col[10]]].std(1)
#             X_train['std6'] = X_train[[ch_col[5],ch_col[11]]].std(1)
#             X_train['std7'] = X_train[[ch_col[12],ch_col[18],ch_col[24],ch_col[30]]].std(1)
#             X_train['std8'] = X_train[[ch_col[13],ch_col[19],ch_col[25],ch_col[31]]].std(1)
#             X_train['std9'] = X_train[[ch_col[14],ch_col[20],ch_col[26],ch_col[32]]].std(1)
#             X_train['std10'] = X_train[[ch_col[15],ch_col[21],ch_col[27],ch_col[33]]].std(1)
#             X_train['std11'] = X_train[[ch_col[16],ch_col[22],ch_col[28],ch_col[34]]].std(1)
#             X_train['std12'] = X_train[[ch_col[17],ch_col[23],ch_col[29],ch_col[35]]].std(1)
#             X_train['std13'] = X_train[[ch_col[36],ch_col[42],ch_col[48],ch_col[54],ch_col[60],ch_col[66],ch_col[72],ch_col[78]]].std(1)
#             X_train['std14'] = X_train[[ch_col[37],ch_col[43],ch_col[49],ch_col[55],ch_col[61],ch_col[67],ch_col[73],ch_col[79]]].std(1)
#             X_train['std15'] = X_train[[ch_col[38],ch_col[44],ch_col[50],ch_col[56],ch_col[62],ch_col[68],ch_col[74],ch_col[80]]].std(1)
#             X_train['std16'] = X_train[[ch_col[39],ch_col[45],ch_col[51],ch_col[57],ch_col[63],ch_col[69],ch_col[75],ch_col[81]]].std(1)
#             X_train['std17'] = X_train[[ch_col[40],ch_col[46],ch_col[52],ch_col[58],ch_col[64],ch_col[70],ch_col[76],ch_col[82]]].std(1)
#             X_train['std18'] = X_train[[ch_col[41],ch_col[47],ch_col[53],ch_col[59],ch_col[65],ch_col[71],ch_col[77],ch_col[83]]].std(1)
            
#             X_train['std_max_col'] = 0
#             X_train['std_min_col'] = 0
            
#             tm_l = list(X_train.loc[new_id,'std1':'std18'])
#             X_train.loc[new_id,'std_max_col'] = tm_l.index(max(tm_l))
#             X_train.loc[new_id,'std_min_col'] = tm_l.index(min(tm_l))

            total_shift_data = pd.concat([total_shift_data,X_train])
            print(total_shift_data.shape)
        new_id+=1

c:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.17it/s]


label,  37 진행중
(1, 361)
(2, 361)
(3, 361)
(4, 361)
(5, 361)
(6, 361)
(7, 361)


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.05it/s]

(8, 361)


...

(192, 361)


(193, 361)
(194, 361)
(195, 361)
(196, 361)
(197, 361)
(198, 361)
(199, 361)


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1041.03it/s]

(200, 361)


(201, 361)
(202, 361)
(203, 361)
(204, 361)


In [17]:
total_shift_data.head()

acc_x                        acc_y                         acc_z  \
           max      min      mean       max       min      mean       max   
id                                                                          
1000  1.344268  0.59194  0.931329  0.176871 -0.624113 -0.218471 -0.054876   
1000  1.344268  0.59194  0.931329  0.176871 -0.624113 -0.218471 -0.054876   
1000  1.344268  0.59194  0.931329  0.176871 -0.624113 -0.218471 -0.054876   
1000  1.344268  0.59194  0.931329  0.176871 -0.624113 -0.218471 -0.054876   
1000  1.344268  0.59194  0.931329  0.176871 -0.624113 -0.218471 -0.054876   

                               gy_x                            gy_y            \
           min      mean        max        min      mean        max       min   
id                                                                              
1000 -0.786336 -0.370422  31.644123 -46.254837 -1.865269  69.847244 -85.88768   
1000 -0.786336 -0.370422  31.644123 -46.254837 -1.865269  69.847244 -85.88768   
1000 -0.786336 -0.370422  31.644123 -46.254837 -1.865269  69.847244 -85.88768   
1000 -0.786336 -0.370422  31.644123 -46.254837 -1.865269  69.847244 -85.88768   
1000 -0.786336 -0.370422  31.644123 -46.254837 -1.865269  69.847244 -85.88768   

                     gy_z                      label  
          mean        max        min      mean        
id                                                    
1000 -3.359506  55.953827 -79.930031  1.182107    37  
1000 -3.359506  55.953827 -79.930031  1.182107    37  
1000 -3.359506  55.953827 -79.930031  1.182107    37  
1000 -3.359506  55.953827 -79.930031  1.182107    37  
1000 -3.359506  55.953827 -79.930031  1.182107    37

In [18]:
total_shift_data.shape

(11404, 19)